In [1]:


import sparknlp
 
spark = sparknlp.start(gpu = True) # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *

from sparknlp.common import *

import sparknlp
from sparknlp.pretrained import PretrainedPipeline
 
print("Spark NLP version", sparknlp.version())
 
print("Apache Spark version:", spark.version)



22/12/09 13:46:12 WARN Utils: Your hostname, Vader resolves to a loopback address: 127.0.1.1; using 192.168.1.94 instead (on interface enp5s0f0)
22/12/09 13:46:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/sjhjrok/anaconda3/envs/NLP/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sjhjrok/.ivy2/cache
The jars for the packages stored in: /home/sjhjrok/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp-gpu_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-aad46afe-b3f0-4626-87b3-7647433f1a31;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-gpu_2.12;4.2.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.project

22/12/09 13:46:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/09 13:46:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Spark NLP version 4.2.2
Apache Spark version: 3.3.1


In [2]:


from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, './eng_custom.train')
testing_data= CoNLL().readDataset(spark, './eng_custom.testa')



In [3]:
import pyspark.sql.functions as F

#training_data.select(F.explode(F.arrays_zip('token.result', 'pos.result',  'label.result')).alias("cols")) \
#.select(F.expr("cols['0']").alias("token"),
#        F.expr("cols['1']").alias("pos"),
#        F.expr("cols['2']").alias("ner_label")).show(truncate=50)


In [4]:
!mkdir ner_logs

mkdir: cannot create directory ‘ner_logs’: File exists


In [5]:


# You can use any word embeddings you want (Glove, Elmo, Bert, custom etc.)
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')
embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

nerTagger = NerDLApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(500)\
      .setLr(0.003)\
      .setBatchSize(32)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setValidationSplit(0.2)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setOutputLogsPath('ner_logs') # if not set, logs will be written to ~/annotator_logs
 #    .setGraphFolder('graphs') >> put your graph file (pb) under this folder if you are using a custom graph generated thru 4.1 NerDL-Graph.ipynb notebook
 #    .setEnableMemoryOptimizer() >> if you have a limited memory and a large conll file, you can set this True to train batch by batch 
    


ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

ner_pipeline = Pipeline(stages=[
      embeddings,
      nerTagger,
      ner_converter
 ])



glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ]glove_100d download started this may take some time.
Approximate size to download 145.3 MB
Download done! Loading the resource.
[OK!]


In [6]:
%%time
ner_model = ner_pipeline.fit(training_data)
ner_model.stages[1].write().overwrite().save('outputs/ner_wiki_glove100d_en')

2022-12-09 13:46:24.123357: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-09 13:46:24.123375: I external/org_tensorflow/tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-09 13:46:24.603137: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-09 13:46:24.603765: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open 

Training started - total epochs: 500 - lr: 0.003 - batch size: 32 - labels: 9 - chars: 24 - training examples: 243
Epoch 1/500 started, lr: 0.003, dataset size: 243
Epoch 1/500 - 0.85s - loss: 48.66227 - batches: 9
Quality on validation dataset (20.0%), validation examples = 48
time to finish evaluation: 0.19s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 5	 9	 2	 0.35714287	 0.71428573	 0.47619048
Loc	 18	 3	 2	 0.85714287	 0.9	 0.87804884
Act	 27	 0	 19	 1.0	 0.5869565	 0.739726
0	 32	 13	 3	 0.7111111	 0.9142857	 0.8
Obj	 21	 4	 3	 0.84	 0.875	 0.85714287
Trigger	 14	 0	 0	 1.0	 1.0	 1.0
tp: 117 fp: 29 fn: 29 labels: 6
Macro-average	 prec: 0.79423285, rec: 0.8317547, f1: 0.8125608
Micro-average	 prec: 0.80136985, rec: 0.80136985, f1: 0.80136985
Epoch 2/500 started, lr: 0.0029850747, dataset size: 243
Epoch 2/500 - 0.19s - loss: 32.255707 - batches: 9
Quality on validation dataset (20.0%), validation examples = 48
time to finish evaluation: 0.02s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 0	 0	 7

Epoch 14/500 - 0.16s - loss: 1.3376471 - batches: 9
Quality on validation dataset (20.0%), validation examples = 48
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 7	 0	 0	 1.0	 1.0	 1.0
Loc	 19	 0	 1	 1.0	 0.95	 0.9743589
Act	 44	 0	 2	 1.0	 0.95652175	 0.9777778
0	 35	 1	 0	 0.9722222	 1.0	 0.9859155
Obj	 24	 2	 0	 0.9230769	 1.0	 0.96000004
Trigger	 14	 0	 0	 1.0	 1.0	 1.0
tp: 143 fp: 3 fn: 3 labels: 6
Macro-average	 prec: 0.9825499, rec: 0.9844203, f1: 0.9834842
Micro-average	 prec: 0.9794521, rec: 0.9794521, f1: 0.9794521
Epoch 15/500 started, lr: 0.0028037382, dataset size: 243
Epoch 15/500 - 0.16s - loss: 1.0519698 - batches: 9
Quality on validation dataset (20.0%), validation examples = 48
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 7	 0	 0	 1.0	 1.0	 1.0
Loc	 20	 0	 0	 1.0	 1.0	 1.0
Act	 44	 0	 2	 1.0	 0.95652175	 0.9777778
0	 35	 1	 0	 0.9722222	 1.0	 0.9859155
Obj	 24	 1	 0	 0.96	 1.0	 0.9795918
Trigger	 14	 0	 0	 1.0	 1.0	 1

Epoch 29/500 - 0.16s - loss: 0.096609786 - batches: 9
Quality on validation dataset (20.0%), validation examples = 48
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 7	 0	 0	 1.0	 1.0	 1.0
Loc	 20	 0	 0	 1.0	 1.0	 1.0
Act	 43	 0	 3	 1.0	 0.9347826	 0.96629214
0	 35	 2	 0	 0.9459459	 1.0	 0.97222215
Obj	 24	 1	 0	 0.96	 1.0	 0.9795918
Trigger	 14	 0	 0	 1.0	 1.0	 1.0
tp: 143 fp: 3 fn: 3 labels: 6
Macro-average	 prec: 0.9843243, rec: 0.98913044, f1: 0.9867215
Micro-average	 prec: 0.9794521, rec: 0.9794521, f1: 0.9794521
Epoch 30/500 started, lr: 0.0026200875, dataset size: 243
Epoch 30/500 - 0.16s - loss: 0.22840191 - batches: 9
Quality on validation dataset (20.0%), validation examples = 48
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 7	 0	 0	 1.0	 1.0	 1.0
Loc	 20	 0	 0	 1.0	 1.0	 1.0
Act	 43	 0	 3	 1.0	 0.9347826	 0.96629214
0	 35	 2	 0	 0.9459459	 1.0	 0.97222215
Obj	 24	 1	 0	 0.96	 1.0	 0.9795918
Trigger	 14	 0	 0	 1.0	 1.0	 1.0
tp: 

Epoch 43/500 - 0.16s - loss: 0.101318255 - batches: 9
Quality on validation dataset (20.0%), validation examples = 48
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 7	 2	 0	 0.7777778	 1.0	 0.875
Loc	 19	 0	 1	 1.0	 0.95	 0.9743589
Act	 43	 0	 3	 1.0	 0.9347826	 0.96629214
0	 33	 2	 2	 0.94285715	 0.94285715	 0.94285715
Obj	 24	 2	 0	 0.9230769	 1.0	 0.96000004
Trigger	 14	 0	 0	 1.0	 1.0	 1.0
tp: 140 fp: 6 fn: 6 labels: 6
Macro-average	 prec: 0.9406187, rec: 0.97127336, f1: 0.9557003
Micro-average	 prec: 0.9589041, rec: 0.9589041, f1: 0.9589041
Epoch 44/500 started, lr: 0.0024691357, dataset size: 243
Epoch 44/500 - 0.16s - loss: 0.039761234 - batches: 9
Quality on validation dataset (20.0%), validation examples = 48
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 7	 2	 0	 0.7777778	 1.0	 0.875
Loc	 19	 0	 1	 1.0	 0.95	 0.9743589
Act	 43	 0	 3	 1.0	 0.9347826	 0.96629214
0	 33	 2	 2	 0.94285715	 0.94285715	 0.94285715
Obj	 24	 2	 0	 0.923

...

Epoch 465/500 - 0.16s - loss: 0.0024841046 - batches: 9
Quality on validation dataset (20.0%), validation examples = 48
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 7	 2	 0	 0.7777778	 1.0	 0.875
Loc	 19	 0	 1	 1.0	 0.95	 0.9743589
Act	 43	 0	 3	 1.0	 0.9347826	 0.96629214
0	 33	 2	 2	 0.94285715	 0.94285715	 0.94285715
Obj	 24	 2	 0	 0.9230769	 1.0	 0.96000004
Trigger	 14	 0	 0	 1.0	 1.0	 1.0
tp: 140 fp: 6 fn: 6 labels: 6
Macro-average	 prec: 0.9406187, rec: 0.97127336, f1: 0.9557003
Micro-average	 prec: 0.9589041, rec: 0.9589041, f1: 0.9589041
Epoch 466/500 started, lr: 9.0225565E-4, dataset size: 243
Epoch 466/500 - 0.16s - loss: 2.0856327E-4 - batches: 9
Quality on validation dataset (20.0%), validation examples = 48
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 7	 2	 0	 0.7777778	 1.0	 0.875
Loc	 20	 0	 0	 1.0	 1.0	 1.0
Act	 44	 0	 2	 1.0	 0.95652175	 0.9777778
0	 33	 1	 2	 0.9705882	 0.94285715	 0.95652175
Obj	 24	 1	 0	 0.96	 1.

Epoch 479/500 - 0.16s - loss: 0.111094646 - batches: 9
Quality on validation dataset (20.0%), validation examples = 48
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 7	 2	 0	 0.7777778	 1.0	 0.875
Loc	 20	 0	 0	 1.0	 1.0	 1.0
Act	 44	 0	 2	 1.0	 0.95652175	 0.9777778
0	 33	 1	 2	 0.9705882	 0.94285715	 0.95652175
Obj	 24	 1	 0	 0.96	 1.0	 0.9795918
Trigger	 14	 0	 0	 1.0	 1.0	 1.0
tp: 142 fp: 4 fn: 4 labels: 6
Macro-average	 prec: 0.9513943, rec: 0.9832298, f1: 0.96705014
Micro-average	 prec: 0.9726027, rec: 0.9726027, f1: 0.9726027
Epoch 480/500 started, lr: 8.836524E-4, dataset size: 243
Epoch 480/500 - 0.16s - loss: 3.7229725E-4 - batches: 9
Quality on validation dataset (20.0%), validation examples = 48
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 7	 2	 0	 0.7777778	 1.0	 0.875
Loc	 19	 0	 1	 1.0	 0.95	 0.9743589
Act	 43	 0	 3	 1.0	 0.9347826	 0.96629214
0	 33	 2	 2	 0.94285715	 0.94285715	 0.94285715
Obj	 24	 2	 0	 0.9230769	 1.0	 

Epoch 493/500 - 0.16s - loss: 0.002654155 - batches: 9
Quality on validation dataset (20.0%), validation examples = 48
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 7	 2	 0	 0.7777778	 1.0	 0.875
Loc	 20	 0	 0	 1.0	 1.0	 1.0
Act	 40	 0	 6	 1.0	 0.8695652	 0.9302325
0	 33	 4	 2	 0.8918919	 0.94285715	 0.9166667
Obj	 24	 1	 0	 0.96	 1.0	 0.9795918
Pos	 0	 1	 0	 0.0	 0.0	 0.0
Trigger	 14	 0	 0	 1.0	 1.0	 1.0
tp: 138 fp: 8 fn: 8 labels: 7
Macro-average	 prec: 0.8042385, rec: 0.83034605, f1: 0.8170838
Micro-average	 prec: 0.94520545, rec: 0.94520545, f1: 0.94520545
Epoch 494/500 started, lr: 8.658009E-4, dataset size: 243
Epoch 494/500 - 0.16s - loss: 0.011922945 - batches: 9
Quality on validation dataset (20.0%), validation examples = 48
time to finish evaluation: 0.01s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 7	 2	 0	 0.7777778	 1.0	 0.875
Loc	 20	 0	 0	 1.0	 1.0	 1.0
Act	 41	 0	 5	 1.0	 0.8913044	 0.9425287
0	 33	 3	 2	 0.9166667	 0.94285715	 0.92957747
Obj	 24	 1	 

CPU times: user 275 ms, sys: 87.1 ms, total: 363 ms
Wall time: 1min 32s


2022-12-09 13:47:54.444018: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

In [7]:

#predictions = ner_model.transform(testing_data) 
#predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
#           .select(F.expr("cols['0']").alias("token"),
#                   F.expr("cols['1']").alias("ground_truth"),
#                   F.expr("cols['2']").alias("prediction")).show(truncate=False)

In [8]:
from sklearn.metrics import classification_report
 
#preds_df = predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
#                      .select(F.expr("cols['0']").alias("token"),
#                              F.expr("cols['1']").alias("ground_truth"),
#                              F.expr("cols['2']").alias("prediction")).toPandas()
 
#print(classification_report(preds_df['ground_truth'], preds_df['prediction']))

In [9]:
#import shutil

#shutil.make_archive("/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en", 'zip', "/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en")

In [10]:
text_list = [
    """Navagate to kitchen"""
]


In [11]:
loaded_ner_model = NerDLModel.load("outputs/ner_wiki_glove100d_en")\
   .setInputCols(["sentence", "token", "embeddings"])\
   .setOutputCol("ner")

load_ner_pipeline = Pipeline(stages=[
      document_assembler,
      tokenizer,
      embeddings,
      loaded_ner_model,
      ner_converter
 ])


2022-12-09 13:47:55.254720: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

In [12]:
from sparknlp.base import LightPipeline

text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = load_ner_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['ner'])

{'document': ['navigate to kitchen'], 'ner_chunk': ['navigate to kitchen'], 'token': ['navigate', 'to', 'kitchen'], 'ner': ['Act', '0', 'Loc'], 'embeddings': ['navigate', 'to', 'kitchen']}
['Act', '0', 'Loc']


In [13]:
light.annotate("Navigate to kitchen")


{'document': ['Navigate to kitchen'],
 'ner_chunk': ['Navigate to kitchen'],
 'token': ['Navigate', 'to', 'kitchen'],
 'ner': ['Act', '0', 'Loc'],
 'embeddings': ['Navigate', 'to', 'kitchen']}

In [ ]:
text = input("Enter Testing Text\n")
while(text != "exit"):
    results = light.annotate(text)
    print(results['ner'])
    text = input("Enter New Text\n")

Enter Testing Text
ottoman
['Obj']
Enter New Text
test
['Pos']
Enter New Text
yes
['Pos']
Enter New Text
no 
['Neg']
Enter New Text
hello
['Trigger']
Enter New Text
what
['Rqt']
Enter New Text
do
['Act']
Enter New Text
where
['Rqt']
Enter New Text
am 
['Loc']
Enter New Text
I
['0']
Enter New Text
sit on ottoman
['Act', 'Act', 'Obj']
Enter New Text
take out trash
['0', 'Act', 'Act']
Enter New Text
take out garbage
['0', '0', 'Act']
Enter New Text
garbage
['Act']


In [ ]:
#empty_df = spark.createDataFrame([['']]).toDF('text')
#pipeline_model = ner_pipeline.fit(empty_df)
#df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
#result = pipeline_model.transform(df)

In [ ]:




#from sparknlp_display import NerVisualizer



#result = predictions

#NerVisualizer().display(
#    result = result.collect()[0],
#    label_col = 'ner_chunk',
#    document_col = 'document'
#)




